<a href="https://colab.research.google.com/github/tfrizza/DALL-E-tf/blob/main/tfFlowers_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q tensorflow_addons

     |████████████████████████████████| 706kB 6.4MB/s 


In [2]:
!git clone https://github.com/tfrizza/DALL-E-tf.git

Cloning into 'DALL-E-tf'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 75 (delta 39), reused 38 (delta 16), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [3]:
%cd DALL-E-tf

/content/DALL-E-tf


In [45]:
import tensorflow as tf
from tensorflow.keras import Model, mixed_precision
from tensorflow.keras.losses import Loss, MeanSquaredError, MeanAbsoluteError, MSE, MAE
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.callbacks import Callback

import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow_probability import layers as tfpl

import tensorflow_datasets as tfds
from tensorflow_addons.optimizers import LAMB, AdamW

from dall_e_tf.encoder import dvae_encoder
from dall_e_tf.decoder import dvae_decoder
from dall_e_tf.vae import dVAE
from dall_e_tf.losses import LatentLoss
from dall_e_tf.utils import plot_reconstructions

import numpy as np
import attr
from functools import partial

mixed_precision.set_global_policy('float32')
AUTOTUNE = tf.data.AUTOTUNE

In [147]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)
NUM_DEVICES = strategy.num_replicas_in_sync
print("REPLICAS: ", NUM_DEVICES)

Running on TPU  ['10.51.112.18:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.51.112.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.51.112.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [148]:
def crop(image):
    y_nonzero, x_nonzero, _ = tf.experimental.numpy.nonzero(image)
    return image[tf.reduce_min(y_nonzero):tf.reduce_max(y_nonzero), tf.reduce_min(x_nonzero):tf.reduce_max(x_nonzero)]

def preprocess(data, h=128, w=128):
    img = crop(data['image'])
    img = tf.image.resize(img, size=(h,w), antialias=False)
    img /= 255
    return img

GLOBAL_BATCH_SIZE = 16 * NUM_DEVICES

train_dataset = tfds.load('tf_flowers', 
                            split='train', 
                            shuffle_files=True,
                            try_gcs=True
                          )
train_dataset = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)\
                             .batch(GLOBAL_BATCH_SIZE)\
                             .prefetch(buffer_size=AUTOTUNE)

train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
train_dataset

<PrefetchDataset shapes: (None, 128, 128, 3), types: tf.float32>

In [149]:
class LatentLoss(Loss):

    def call(self, dummy_ground_truth, outputs):
        del dummy_ground_truth
        z_e, z_q = tf.split(outputs, 2, axis=-1)
        vq_loss = tf.reduce_mean(tf.square(tf.stop_gradient(z_e) - z_q))
        commit_loss = tf.reduce_mean(tf.square(z_e - tf.stop_gradient(z_q)))
        return vq_loss + 1.0 * commit_loss

In [150]:
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops

# class TemperatureScheduler(Callback):
#     def __init__(self, schedule, layer_name='gumbel-softmax', verbose=0):
#         super(TemperatureScheduler, self).__init__()
#         self.schedule = schedule
#         self.layer_name = layer_name
#         self.verbose = verbose

#     def on_epoch_begin(self, epoch, logs=None):
#         layer = self.model.get_layer(self.layer_name)
#         if not hasattr(layer, '_most_recently_built_distribution'):
#             raise ValueError('Layer must have a "_most_recently_built_distribution" attribute.')
#         distrib = layer._most_recently_built_distribution
#         if not hasattr(distrib, 'temperature'):
#             raise ValueError('Distribution must have a "temperature" attribute.')
#         # T = float(K.get_value(distrib.temperature))
#         # T = distrib.temperature
#         T = self.schedule(epoch)
#         if not isinstance(T, (ops.Tensor, float, np.float32, np.float64)):
#             raise ValueError('The output of the "schedule" function '
#                             'should be float.')
#         if isinstance(T, ops.Tensor) and not T.dtype.is_floating:
#             raise ValueError('The dtype of Tensor should be float')
#         K.set_value(distrib.temperature, K.get_value(T))
#         if self.verbose > 0:
#             print('\nEpoch %05d: TemperatureScheduler reducing temperature '
#                 'rate to %s.' % (epoch + 1, T))

#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         T = self.model.get_layer(self.layer_name)._most_recently_built_distribution.temperature
#         logs['temperature'] = K.get_value(T)


class TemperatureScheduler(Callback):
    def __init__(self, schedule, verbose=False):
        super(TemperatureScheduler, self).__init__()
        self.schedule = schedule
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        temperature = self.schedule(epoch)
        # self.model.temperature = temperature
        if self.verbose:
            print(f'Setting temperature to {self.model.temperature}')

In [151]:
vocab_size = 8192//2
n_hid = 256//2

class dVAE(Model):
    def __init__(self, enc, dec, initial_temp=1.0, temp_decay=0.9):
        super(dVAE, self).__init__()
        self.enc = enc
        self.dec = dec
        self.temperature = initial_temp
        self.temp_decay = temp_decay

        self.gumbel_softmax = tfpl.DistributionLambda(
            lambda x: tfd.RelaxedOneHotCategorical(temperature=x[0], logits=x[1]) # Gumbel-softmax
            , name='gumbel-softmax'
        )

    def call(self, inputs, training=False):
        x, temperature = inputs
        z_e = self.enc(x)
        z_q = self.gumbel_softmax([temperature, z_e])

        z_hard = tf.math.argmax(z_e, axis=-1) # non-differentiable
        z_hard = tf.one_hot(z_hard, enc.output.shape[-1], dtype=z_q.dtype)

        z = z_q + tf.stop_gradient(z_hard - z_q) # straight-through Gumbel-softmax
        x_rec = self.dec(z)
        latents = tf.stack([z_hard, z_q], -1, name='latent')
        return x_rec, latents, temperature
    
    def train_step(self, x):
        with tf.GradientTape() as tape:
            x_pred, latents, T = self([x, self.temperature], training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(x, x_pred, regularization_losses=self.losses)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(x, x_pred)
        results = {m.name: m.result() for m in self.metrics}
        results['temperature'] = self.temperature
        return results

with strategy.scope():
    enc = dvae_encoder(group_count=2, n_hid=n_hid, n_blk_per_group=2, input_channels=3, vocab_size=vocab_size, activation='swish')
    dec = dvae_decoder(group_count=2, n_init=n_hid//2, n_hid=n_hid, n_blk_per_group=2, output_channels=3, vocab_size=vocab_size, activation='swish')

    vae = dVAE(enc, dec)

    epochs = 200
    temp_schedule = PolynomialDecay(0.9, epochs, 1/16, 8) # quadratic decay
    lr_schedule = PolynomialDecay(1e-3, epochs, 1e-4, 0.1) # sqrt decay
    optimizer = AdamW(weight_decay=1e-4, learning_rate=lr_schedule)

    def psnr(x1, x2):
        return tf.image.psnr(x1, x2, max_val=1.0)
    vae.compile(loss=['mse', None], optimizer=optimizer, metrics=[psnr])

# vae.build(input_shape=(128,128,128,3))
# vae.summary(line_length=200)

In [152]:
vae.fit(train_dataset,
        # validation_data=x_test,
        epochs=1000,
        callbacks=[TemperatureScheduler(temp_schedule, True)]
        )

Epoch 1/1000
Setting temperature to 1.0
 6/29 [=====>........................] - ETA: 6s - loss: 0.0956 - psnr: 10.5565 - temperature: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0025s vs `on_train_batch_end` time: 3.2613s). Check your callbacks.


29/29 [==============================] - 41s 274ms/step - loss: 0.0872 - psnr: 10.9622 - temperature: 1.0000
Epoch 2/1000
Setting temperature to 1.0
29/29 [==============================] - 8s 275ms/step - loss: 0.0790 - psnr: 11.3681 - temperature: 1.0000
Epoch 3/1000
Setting temperature to 1.0
29/29 [==============================] - 8s 275ms/step - loss: 0.0781 - psnr: 11.4744 - temperature: 1.0000
Epoch 4/1000
Setting temperature to 1.0
29/29 [==============================] - 8s 280ms/step - loss: 0.0574 - psnr: 12.8731 - temperature: 1.0000
Epoch 5/1000
Setting temperature to 1.0
29/29 [==============================] - 8s 280ms/step - loss: 0.0440 - psnr: 13.9833 - temperature: 1.0000
Epoch 6/1000
Setting temperature to 1.0
29/29 [==============================] - 8s 281ms/step - loss: 0.0405 - psnr: 14.3649 - temperature: 1.0000
Epoch 7/1000
Setting temperature to 1.0
29/29 [==============================] - 8s 278ms/step - loss: 0.0311 - psnr: 15.5664 - temperature: 1.0000
Epo

KeyboardInterrupt: ignored

In [153]:
train_batch = next(iter(train_dataset))
plot_reconstructions(vae(train_batch[:10])[0], train_batch[:10])

ValueError: ignored